### Logistic regression from scratch
**We will :**
- Build the general architecture of a learning algorithm, including:
    - Initializing parameters
    - Calculating the cost function and its gradient
    - Using an optimization algorithm (gradient descent) 
- Gather all three functions above into a main model function, in the right order.

## 1 - Packages ##

import all the packages that you will need during this assignment. 
- [numpy](www.numpy.org) is the fundamental package for scientific computing with Python.
- [matplotlib](http://matplotlib.org) is a famous library to plot graphs in Python.
* [scikit-learn](http://scikit-learn.org/stable/) a library with Simple and efficient tools for data mining and data analysis

In [105]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

%matplotlib inline
np.random.seed(42)

## 2 - Dataset ##

we will use the make_classification data from sklearn

Loading the data by with the  following code.

In [290]:
X,Y=datasets.make_classification(n_samples=100000, n_features=100,
                                    n_informative=100,n_classes=2, n_redundant=0,
                                    random_state=42)



## - Data-split  ##

we will split the data with the following distribution 
- 99% -training set
- 1% -test set

we will use the sklearn train_test_split

In [291]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.01,
                                                    random_state=42)

For convenience, we reshape the data into  a numpy-array of shape (1, m). After this, our training (and test) dataset is a numpy-array where each column represents one training example. There should be m_train (respectively m_test) columns.

In [292]:
# we need to reshape our data to column vectors 
X_sk=X_train
y_sk=y_train
X_train=X_train.reshape(X_train.shape[0],-1).T
X_test=X_test.reshape(X_test.shape[0],-1).T
y_train=y_train.reshape(y_train.shape[0],-1).T
y_test=y_test.reshape(y_test.shape[0],-1).T


## 3 - Building the parts of our algorithm ## 

The main steps for building a Neural Network are:
1. Define the model structure (such as number of input features) 
2. Initialize the model's parameters
3. Loop:
    - Calculate current loss (forward propagation)
    - Calculate current gradient (backward propagation)
    - Update parameters (gradient descent)

You often build 1-3 separately and integrate them into one function we call `model()`.

### 3.1 - Helper functions

**sigmoid**:  implementing `sigmoid()`.  $sigmoid( w^T x + b) = \frac{1}{1 + e^{-(w^T x + b)}}$ to make predictions. 

In [159]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """
    s=1/(1+np.exp(-z))
    return s

### 3.2 - Initializing parameters

**parameter initilization:** Implementing parameter initialization we  have to initialize w as a vector of zeros. I

In [160]:
def parameter_initiliazation(dimension):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias)
    """
    
    w=np.zeros((dimension,1),dtype=float)
    b=0
    return w,b

### 3.3 -forward propagation

**forward propagation:** Implementing forward propagation 

In [161]:
def forward_propagation(X,Y,W,b):
    """
    
    """
    Z=np.dot(W.T,X)+b
    A=sigmoid(Z)
    return A
    

In [162]:
def compute_cost(A,Y,m):
    cost=-(1/m)*np.sum(Y*np.log(A)+(1-Y)*np.log(1-A))
    return cost

In [163]:
def back_propagation(A,X,Y,m):
    dw=1/m*np.dot(X,(A-Y).T)
    db=1/m*np.sum(A-Y)
    return dw,db

In [164]:
def update_parameters(W,b,dw,db,learning_rate):
    W=W-learning_rate*dw
    b=b-learning_rate*db
    return W,b

In [165]:
def predict(W,b,X):
    z=np.dot(W.T,X)+b
    A=sigmoid(z)
    return np.choose(A < 0.5,[1,0])

In [293]:
def model(X,Y,learning_rate=0.1):
    W,b=parameter_initiliazation(X.shape[0])
    m=X.shape[1]
    for i in range(100):
        A=forward_propagation(X,Y,W,b)
        cost=compute_cost(A,Y,m)
        dw,db=back_propagation(A,X,Y,m)
        W,b=update_parameters(W,b,dw,db,learning_rate)
       
    return predict(W,b,X) 

In [294]:
preds=model(X_train,y_train)

In [295]:
accuracy_score(preds.reshape(preds.shape[1],1),y_train.reshape(y_train.shape[1],1))


0.83712121212121215